# Compute contact between residues

In [6]:
from Bio.PDB import PDBParser, NeighborSearch
import numpy as np

def compute_residue_contacts_alpha_CA(pdb_file, output_file, residues_range, cutoff=1.0):
    # Parse the PDB file
    parser = PDBParser()
    structure = parser.get_structure('protein', pdb_file)

    # Collect all Cα atoms
    ca_atoms = [atom for residue in structure.get_residues() for atom in residue if atom.get_name() == 'CA']

    # Define NeighborSearch object for these Cα atoms
    ns = NeighborSearch(ca_atoms)

    # Define the sets of residues you are interested in
    #n_terminal_residues = [residue for residue in structure.get_residues() if residue.id[1] <= cutoff and residue.has_id('CA')]
    n_terminal_residues = [residue for residue in structure.get_residues() if residue.id[1] in residues_range and residue.has_id('CA')]
    #rest_of_protein_residues = [residue for residue in structure.get_residues() if residue.id[1] > 50 and residue.has_id('CA')]
    rest_of_protein_residues = [residue for residue in structure.get_residues() if residue.has_id('CA')]

    # Store residue contacts
    residue_contacts = []
    #cutoff = 100.0  # Define your distance cutoff in Angstroms

    for n_res in n_terminal_residues:
        if n_res.has_id('CA'):
            n_ca = n_res['CA']
            for r_res in rest_of_protein_residues:
                if r_res.has_id('CA'):
                    r_ca = r_res['CA']
                    distance = np.linalg.norm(n_ca.coord - r_ca.coord)  # Calculate Euclidean distance
                    #if distance <= cutoff:
                    residue_contacts.append((n_res, r_res, distance))

    with open(output_file, 'w') as f:
        f.write(f"Contact\tResidue1_Name\tResidue1_Pos\tResidue2_Name\tResidue2_Pos\tPairs\tDistance(Å)\n")
        for i, contact in enumerate(residue_contacts):
            f.write(f"{i+1}\t{contact[0].get_resname()}\t{contact[0].id[1]}\t{contact[1].get_resname()}\t{contact[1].id[1]}\t{(contact[0].id[1],contact[1].id[1])}\t{contact[2]:.2f}\n")

In [7]:
# Extract contact for compact version

# Define the range of residues in the N-terminal region (e.g., 1-100)
n_terminal_range = range(1, 151)  
in_file = '../data/nsP4/nsP4_pdb/nsp4-RdRp_compact_chainX.pdb'
out_file = '../results/Distance/nsP4/nsp4-RdRp_compact_distance_Nter150.txt'
compute_residue_contacts_alpha_CA(in_file, out_file, n_terminal_range )

In [8]:
# # Extract contact for extended version

# # Define the range of residues in the N-terminal region (e.g., 1-100)
# n_terminal_range = range(1, 150)  
# in_file = '../data/nsP4/nsP4_pdb/nsp4-RdRp-SAXS_extended_chainX.pdb'
# out_file = '../results/Distance/nsP4/nsp4-RdRp_extended_distance.txt'
# compute_residue_contacts_alpha_CA(in_file, out_file, cutoff=100.0)

In [9]:
# Define the range of residues in the N-terminal region (e.g., 1-100)
n_terminal_range = range(1, 151)  
in_file = '../data/nsP4/nsP4_pdb/nsp4-SAXS-extended-conf-MDsimulated_V25E91_LUO_chainX.pdb'
out_file = '../results/Distance/nsP4/nsp4-SAXS-extended-conf-MDsimulated_V25E91_LUO_distance_Nter150.txt'
compute_residue_contacts_alpha_CA(in_file, out_file, n_terminal_range)